In [1]:
import os
import requests
import json
import openai
import pickle as pkl
import requests
import json
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd
from time import sleep
from tqdm.auto import tqdm
import numpy as np
import ast
from sentence_transformers import SentenceTransformer

from sklearn.manifold import TSNE
import plotly.express as px


/Users/haskar140/anaconda3/envs/KG/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
with open('azure-configuration.json') as inputfile:
    azureconfig = json.load(inputfile)
openai.api_key = azureconfig['key'] 
openai.api_base = azureconfig['endpoint'] 
openai.api_type = 'azure'
openai.api_version = '2023-05-15' # this may change in the future

deployment_name= azureconfig['deployment_name']

response = openai.Embedding.create(
    input="Your text string goes here",
    engine=deployment_name
)
embeddings = response['data'][0]['embedding']
print(len(embeddings))

1536


In [3]:
df_IAB=pd.read_csv('data/IAB_Evaluation.csv')

In [4]:
iab_labels=df_IAB['IAB_Label'].to_list()

In [5]:
with open('data/ram_tag_list.txt', 'r') as f:
    check=f.readlines()

In [6]:
check=[x.split('\n')[0] for x in check]

In [7]:
maf=check

In [8]:
call=iab_labels+maf

In [9]:
print(call[0:10])

['Hair Care', 'Shaving', 'Motorcycles', "Women's Formal Wear", "Women's Clothing", 'Retail Industry', 'Comedy Events', 'Household Supplies', 'Home Security', 'Polar Travel']


In [10]:
total_embeddings=[]

In [11]:
def create_embeddings(item,total_embeddings):
    deployment_name= azureconfig['deployment_name']

    response = openai.Embedding.create(
        input=item,
        engine=deployment_name
    )
    embeddings = response['data'][0]['embedding']
    total_embeddings.append(embeddings)

In [12]:
for items in tqdm(call):
    try:
        items=items.replace("&", 'and')
    except:
        pass

#only for Chat GPT version
    # if items in scores.keys():
    #     # print('here')
    #     key = list(scores[items].keys())[0]
    #     create_embeddings(key,total_embeddings)
    #     continue

    create_embeddings(items,total_embeddings)

  0%|          | 0/4745 [00:00<?, ?it/s]

100%|██████████| 4745/4745 [10:42<00:00,  7.38it/s]  


In [13]:
len(total_embeddings)

4745

In [14]:
iab_embeddings=total_embeddings[0:160]

In [15]:
maf_embeddings=total_embeddings[160:]

In [16]:
nodes=maf

In [17]:
cos_sim=cosine_similarity(np.array(iab_embeddings[90]).reshape(1,-1),np.array(maf_embeddings[15]).reshape(1,-1))
print(cos_sim)

[[0.80021651]]


In [18]:
with open("data/problematic.pkl", 'rb') as f:
    problematic=pkl.load(f)

In [19]:
with open("data/child_parent_dic.pkl", 'rb') as f:
    child_parent_dic=pkl.load(f)

In [32]:
# top_n_labels_new=[]
# top_n_cosines_new=[]
# top_n_nodes=[]
# top_n_node_ids=[]
# top_n_IAB_Candidates=[]
# top_n_H_scores=[]
# top_n_labels=[]
# top_n_cosines=[]

# tsne_embeddings=[]



# for i in tqdm(range(len(maf_embeddings))):
#     similarities=[]
#     for j in range(len(iab_embeddings)):
#         cos_sim=cosine_similarity(np.array(iab_embeddings[j]).reshape(1,-1),np.array(maf_embeddings[i]).reshape(1,-1))
#         similarities.append([call[160+i],call[j],cos_sim])
    
#     similarities.sort(key=lambda x: x[2], reverse=True)


#     #threshold logic needed
#     #print(similarities[0:10])
#     #similarities=similarities[0:20]

#     similarities=[x for x in similarities if x[2]>=0.765]

#     new_similarities=[]

#     for x in similarities:
#         if x[2]>=0.785 and x[1] in problematic:
#             new_similarities.append(x)
#         elif x[2]>=0.765 and x[1] not in problematic:
#             new_similarities.append(x)
#         else:
#             continue
    
        
#     if len(new_similarities)==0:
#         new_similarities.append(['No_Pred','No_Pred',[[0]]])
    
#     temp_iab_labels=[]
#     temp_cosines=[]


#     for items in new_similarities:
#         tsne_embeddings.append(maf_embeddings[i])
#         #print(items[1])
#         temp_iab_labels.append(items[1])
#         temp_cosines.append(items[2][0])
        
#         top_n_labels_new.append(items[1])
#         top_n_cosines_new.append(items[2][0])
#         top_n_nodes.append(nodes[i])
#         #top_n_node_ids.append(node_id[i])
#         #top_n_IAB_Candidates.append(IAB_Candidates[i])
#         #top_n_H_scores.append(H_Mappings[i])


    
#     top_n_labels.append(temp_iab_labels)
#     top_n_cosines.append(temp_cosines)

top_n_labels_new=[]
top_n_cosines_new=[]
top_n_nodes=[]
top_n_node_ids=[]
top_n_IAB_Candidates=[]
top_n_H_scores=[]
top_n_labels=[]
top_n_cosines=[]

tsne_embeddings=[]



for i in tqdm(range(len(maf_embeddings))):
    similarities=[]
    for j in range(len(iab_embeddings)):
        cos_sim=cosine_similarity(np.array(iab_embeddings[j]).reshape(1,-1),np.array(maf_embeddings[i]).reshape(1,-1))
        similarities.append([call[160+i],call[j],cos_sim])
    
    similarities.sort(key=lambda x: x[2], reverse=True)


    #threshold logic needed
    #print(similarities[0:10])
    #similarities=similarities[0:20]
    
    #low initial threshold
    similarities=[x for x in similarities if x[2]>=0.70]
    new_similarities=[]

    # new_similarities=[x for x in similarities if x[2]>=0.8065]

    # if len(new_similarities)==0:
    #     new_similarities.append(['No_pred','No_pred',[[0]]])
        
    

    # #higher threshold for problematic keywords, high threshold for general keywords for accuracy
    for x in similarities:
        if len(new_similarities)>=15 and x[2]<0.82:
            break
        if x[2]>=0.8175 and x[1] in problematic:
            new_similarities.append(x)
        elif x[2]>=0.7975 and x[1] not in problematic:
            new_similarities.append(x)
        else:
            continue
    
    #adding atleast 1 adword
    if len(new_similarities)==0:
        #print(call[160+i])
        while len(new_similarities)<1:
            new_similarities.append(similarities[len(new_similarities)])
        

    #print(len(new_similarities))

    #force adding all parent adwords if child is predicted
    new_sim_adwords=[]
    for items in new_similarities:
        new_sim_adwords.append(items[1])

    count=0
    for items in new_similarities:
        if count==5:
            break
        try:
            parent_list=child_parent_dic[items[1]]
            for parent in parent_list:
                if parent not in new_sim_adwords:
                    new_sim_adwords.append(parent)
                    j=call.index(parent)
                    cos_sim=cosine_similarity(np.array(iab_embeddings[j]).reshape(1,-1),np.array(maf_embeddings[i]).reshape(1,-1))
                    if cos_sim>0.765:
                        new_similarities.append([call[160+i],call[j],cos_sim])
        except Exception as e:
            #print(e)
            pass

        count+=1
    

    #new_similarities.sort(key=lambda x: x[2], reverse=True)
     

    #print(len(new_similarities))
        
    temp_iab_labels=[]
    temp_cosines=[]

    count=0
    for items in new_similarities:
        if count>=15 and items[2][0][0]<0.82:
            break
        tsne_embeddings.append(maf_embeddings[i])
        #print(items[1])
        temp_iab_labels.append(items[1])
        temp_cosines.append(items[2][0])
        
        top_n_labels_new.append(items[1])
        top_n_cosines_new.append(items[2][0][0])
        top_n_nodes.append(nodes[i])
        #top_n_node_ids.append(node_id[i])
        #top_n_IAB_Candidates.append(IAB_Candidates[i])
        #top_n_H_scores.append(H_Mappings[i])

        count+=1

    
    top_n_labels.append(temp_iab_labels)
    top_n_cosines.append(temp_cosines)

  7%|▋         | 336/4585 [00:29<05:01, 14.08it/s]

In [ ]:
len(top_n_labels_new)

21814

In [ ]:
dic={'item':top_n_nodes, 
'predictions': top_n_labels_new, 'cosine_predicted': top_n_cosines_new}

In [ ]:
df_MAF_new=pd.DataFrame.from_dict(dic)

In [ ]:
df_MAF_new.to_csv('results/RAM_results_0.80_TSNE_Mapping.csv')

In [ ]:
count=0
for items in tqdm(maf):
    count+=df_MAF_new[df_MAF_new['item']==items].shape[0]

print(count/len(maf))

100%|██████████| 4585/4585 [00:06<00:00, 760.04it/s]

4.757688113413304


In [ ]:
counts=[]
for items in tqdm(iab_labels):
    lol=df_MAF_new[df_MAF_new['predictions']==items].shape[0]
    counts.append(lol)

dic={'IAB_Label':iab_labels, 'Count':counts}

df_adwords=pd.DataFrame.from_dict(dic)
df_adwords.to_csv('results/Adwords_Statistics.csv',index=False)

100%|██████████| 160/160 [00:01<00:00, 95.09it/s]


In [ ]:
iab_labels[counts.index(max(counts))]

'Shopping'

In [ ]:
iab_labels[counts.index(min(counts))]

'Healthcare Industry'

In [ ]:
sum(counts)/160

1373.0625

In [502]:
print(count/len(maf))

47.91712104689204


In [476]:
df_tiers=pd.read_csv('data/MAFlabel_to_Adword_mappings_202210_combined_mappings.csv')

In [477]:
df_tiers=df_tiers['adword']

In [478]:
tier1=[]
all_tiers=df_tiers.to_list()
for tier in all_tiers:
    try:
        tier1.append(tier.split('|')[0])
    except:
        tier1.append(tier)

tier1=list(set(tier1))

In [503]:
len(tier1)

26

In [480]:
diff_Tier=['Events and Attractions', 'Fashion Events','Malls & Shopping Centers', 'Museums & Galleries','Outdoor Activities','Parks & Nature','Awards Shows','Personal Celebrations & Life Events','Wedding','Birthday','Graduation','Prom','Religious Events','Theater Venues and Events','Zoos & Aquariums','Bars & Restaurants','Casinos & Gambling','Cinemas and Events','Comedy Events','Concerts & Music Events']

In [481]:
diff_Tier=['Cats'] #,'Flower Shopping','Grocery Shopping','Holiday Shopping','Household Supplies',"Children's Games and Toys"]

In [482]:
dic={'embeddings':tsne_embeddings, 'labels':top_n_labels_new, 'item': top_n_nodes}
df_labels_embeddings=pd.DataFrame.from_dict(dic)

In [483]:
df_labels_embeddings.head

<bound method NDFrame.head of                                               embeddings  \
0      [-0.014068884775042534, -0.0028959729243069887...   
1      [-0.014068884775042534, -0.0028959729243069887...   
2      [-0.014068884775042534, -0.0028959729243069887...   
3      [-0.014068884775042534, -0.0028959729243069887...   
4      [-0.014068884775042534, -0.0028959729243069887...   
...                                                  ...   
94908  [-0.016872776672244072, -0.017931459471583366,...   
94909  [-0.016872776672244072, -0.017931459471583366,...   
94910  [-0.016872776672244072, -0.017931459471583366,...   
94911  [-0.016872776672244072, -0.017931459471583366,...   
94912  [-0.016872776672244072, -0.017931459471583366,...   

                       labels             item  
0                Video Gaming  3D CG rendering  
1      Technology & Computing  3D CG rendering  
2                 Landscaping  3D CG rendering  
3                    Fine Art  3D CG rendering  
4   

In [484]:
results={}
for label in tqdm(iab_labels):
    mask = df_labels_embeddings['labels'].isin([label])
    df_tier1 = df_labels_embeddings[mask]
    if df_tier1.shape[0]//2<50:
        perp=df_tier1.shape[0]//2
    else:
        perp=50
    to_TSNE_list=df_tier1['embeddings'].to_list()
    to_TSNE_arr=np.array(to_TSNE_list)
    tsne=TSNE(n_components=2, random_state=42, perplexity=perp)
    X_embedded = tsne.fit_transform(to_TSNE_arr)
    results[label]=tsne.kl_divergence_


100%|██████████| 160/160 [04:34<00:00,  1.72s/it]


In [489]:
count=0
#problematic=[]
new_problematic=[]
for k,v in results.items():
    if k not in problematic and v>1.5:
        print(k,v)
        new_problematic.append(k)

Cruises 1.838929295539856
Landscaping 1.6563351154327393
Outdoor Activities 1.8130756616592407
Holiday Shopping 1.5232917070388794
Smartphones 1.686220407485962
Rail Travel 1.6676044464111328
Reptiles 1.5045255422592163
Home & Garden 1.5296838283538818
Sedan 1.5181704759597778
Laptops 1.7031782865524292
Parenting 1.79671049118042
Pets 1.5766956806182861
Games and Puzzles 1.562029480934143
Apartments 1.6987037658691406
Houses 1.8157498836517334
Dining Out 1.5472736358642578
Television 1.7534314393997192
Automotive 1.7090203762054443
Dogs 1.7171697616577148


In [490]:
problematic=problematic+new_problematic

In [515]:
with open("data/problematic.pkl", 'wb') as f:
    pkl.dump(problematic,f)

In [491]:
problematic

['Shaving',
 'Cooking',
 'Coupe',
 'Dating',
 'Wedding',
 'Minivan',
 'Travel',
 'Prom',
 'Science',
 'Convertible',
 'Camping',
 'SUV',
 'Birds',
 'Sports',
 'Shopping',
 'Beauty',
 'Graduation',
 'Gardening',
 'Bath and Shower',
 'Wellness',
 'Station Wagon',
 'Desktops',
 'Movies',
 'Education',
 'Cats',
 'Cruises',
 'Landscaping',
 'Outdoor Activities',
 'Holiday Shopping',
 'Smartphones',
 'Rail Travel',
 'Reptiles',
 'Home & Garden',
 'Sedan',
 'Laptops',
 'Parenting',
 'Pets',
 'Games and Puzzles',
 'Apartments',
 'Houses',
 'Dining Out',
 'Television',
 'Automotive',
 'Dogs']

In [504]:
mask = df_labels_embeddings['labels'].isin(tier1)
df_tier1 = df_labels_embeddings[mask]

In [505]:
df_tier1.head

<bound method NDFrame.head of                                               embeddings  \
0      [-0.014068884775042534, -0.0028959729243069887...   
1      [-0.014068884775042534, -0.0028959729243069887...   
3      [-0.014068884775042534, -0.0028959729243069887...   
4      [-0.014068884775042534, -0.0028959729243069887...   
9      [-0.003964775707572699, 0.003176236292347312, ...   
...                                                  ...   
94894  [-0.016872776672244072, -0.017931459471583366,...   
94899  [-0.016872776672244072, -0.017931459471583366,...   
94900  [-0.016872776672244072, -0.017931459471583366,...   
94901  [-0.016872776672244072, -0.017931459471583366,...   
94902  [-0.016872776672244072, -0.017931459471583366,...   

                         labels             item  
0                  Video Gaming  3D CG rendering  
1        Technology & Computing  3D CG rendering  
3                      Fine Art  3D CG rendering  
4                   Real Estate  3D CG render

In [506]:
df_tier1.shape

(20445, 3)

In [507]:
to_TSNE_list=df_tier1['embeddings'].to_list()

In [508]:
to_TSNE_arr=np.array(to_TSNE_list)

In [451]:
if df_tier1.shape[0]//2<50:
    perp=df_tier1.shape[0]//2
else:
    perp=50

In [509]:
tsne=TSNE(n_components=2, random_state=42, perplexity= perp) #df_tier1.shape[0]//2)
X_embedded = tsne.fit_transform(to_TSNE_arr)
tsne.kl_divergence_

[t-SNE] Computing 151 nearest neighbors...
[t-SNE] Indexed 20445 samples in 0.019s...
[t-SNE] Computed neighbors for 20445 samples in 9.903s...
[t-SNE] Computed conditional probabilities for sample 1000 / 20445
[t-SNE] Computed conditional probabilities for sample 2000 / 20445
[t-SNE] Computed conditional probabilities for sample 3000 / 20445
[t-SNE] Computed conditional probabilities for sample 4000 / 20445
[t-SNE] Computed conditional probabilities for sample 5000 / 20445
[t-SNE] Computed conditional probabilities for sample 6000 / 20445
[t-SNE] Computed conditional probabilities for sample 7000 / 20445
[t-SNE] Computed conditional probabilities for sample 8000 / 20445
[t-SNE] Computed conditional probabilities for sample 9000 / 20445
[t-SNE] Computed conditional probabilities for sample 10000 / 20445
[t-SNE] Computed conditional probabilities for sample 11000 / 20445
[t-SNE] Computed conditional probabilities for sample 12000 / 20445
[t-SNE] Computed conditional probabilities for sa

0.9048817753791809

In [510]:
df_embeddings = pd.DataFrame(X_embedded)
df_embeddings = df_embeddings.rename(columns={0:'x',1:'y'})
df_embeddings = df_embeddings.assign(label=df_tier1.labels.values)
df_embeddings = df_embeddings.assign(items=df_tier1.item.values)

In [511]:
df_embeddings.head()

,x,y,label,items
0,31.391525,112.975906,Video Gaming,3D CG rendering
1,31.391525,112.975906,Technology & Computing,3D CG rendering
2,31.391525,112.975906,Fine Art,3D CG rendering
3,31.391525,112.975906,Real Estate,3D CG rendering
4,85.205971,-49.478771,Video Gaming,3D glasses


In [512]:
fig = px.scatter(
    df_embeddings, x='x', y='y',
    color='label', labels={'color': 'label'},
    hover_data=['items'], title = 'Tier1 Adwords HTML')
fig.show()

In [513]:
fig.write_html("Tier1.html")
